In [1]:
import numpy as np

## Метод Ньютона

In [2]:
def calc_deriv(f):
    return f[:-1] * np.arange(len(f) - 1, 0, -1)

def newton_method(x0, eps, f):
    k = 1
    
    xk = x0
    xk_next = xk - (np.poly1d(f)(xk) / np.poly1d(calc_deriv(f))(xk)) 
    
    while abs(xk_next - xk) > eps:
        xk = xk_next
        xk_next = xk - (np.poly1d(f)(xk) / np.poly1d(calc_deriv(f))(xk))
        k += 1
    
    return {'x': xk_next, 'depth': k}

def solve_with_newton(eps, f):
    for x in sorted(f, reverse=True):
        response = newton_method(x, eps, f)
        if np.isclose(np.poly1d(f)(response['x']), 0., atol=eps):
            yield response | {'x0' : x}

## Упрощенный метод Ньютона

In [3]:
def simplify_newton_method(x0, eps, f):
    k = 1
    
    xk = x0
    f0_deriv = np.poly1d(calc_deriv(f))
    xk_next = xk - (np.poly1d(f)(xk) / f0_deriv(x0))
    
    func = np.poly1d(f)
    
    while abs(func(xk_next)) > eps:
        xk = xk_next
        xk_next = xk - (func(xk) / f0_deriv(x0))
        k += 1
        
    return {'x': xk_next, 'depth': k}

def solve_with_simplified_newton(eps, f):
    for x in sorted(f, reverse=True):
        response = simplify_newton_method(x, eps, f)
        if np.isclose(np.poly1d(f)(response['x']), 0., atol=eps):
            yield response | {'x0' : x}


## Метод Ньютона-Бройдена

In [4]:
def newton_broyden_method(x0, eps, f):
    k = 1
    ck = 1
    
    xk = x0
    xk_next = xk - ck * (np.poly1d(f)(xk) / np.poly1d(calc_deriv(f))(xk))
    
    delta = abs(xk_next - xk)
    while delta > eps:
        if abs(np.poly1d(f)(xk)) >  abs(np.poly1d(f)(xk_next)):
            ck = 0.9
        else:
            ck = 1.2
        xk = xk_next
        xk_next = xk - ck * (np.poly1d(f)(xk) / np.poly1d(calc_deriv(f))(xk))
#         if abs(xk_next - xk) / delta > 0.5:
#             ck = 2
#         elif abs(np.poly1d(f)(xk)) > abs():
#             ck = 1
#         else:
#             ck = 0.7
        delta = abs(xk_next - xk)
        k += 1
        
    return {'x': xk_next, 'depth': k}

## Метод секущих

$$ x^{(k+1)} = x^{(k)} - \frac{f(x^{(k)})}{f(x^{(k)}) - f(x^{(k-1)})}\cdot(x^{(k)} - x^{(k-1)}) $$

In [5]:
def secant_method(x0, eps, delta, f):
    k = 1
    func = np.poly1d(f)
    
    xk_prev = x0
    xk = x0 - (func(x0) / ((func(x0) - func(x0 - delta)) / delta))
    xk_next = xk - (func(xk) / (func(xk) - func(xk_prev))) * (xk - xk_prev)
    
    while abs(xk_next - xk) > eps:
        xk_prev = xk
        xk = xk_next
        xk_next = xk - (func(xk) / (func(xk) - func(xk_prev))) * (xk - xk_prev)
        
        k += 1
    
    return {'x': xk_next, 'depth': k}

### Пример работы
$$ 3x^4 - 4x^3 - 8x^2 + 10x - 7 = 0 $$

In [6]:
eps = 1e-3
delta = 1e-1
coeff = np.array([3, -4, -8, 10, -7])
response = list(solve_with_newton(eps, coeff))[1]
print(f"Метод Ньютона: x = {round(response['x'], 4)}, x0 = {response['x0']}, количество итераций = {response['depth']}")
response = list(solve_with_simplified_newton(eps, coeff))[0]
print(f"Упрощённый метод Ньютона: x = {round(response['x'], 4)}, x0 = {response['x0']}, количество итераций = {response['depth']}")
x0 = 3
response = newton_broyden_method(x0, eps, coeff)
print(f"Метод Ньютона-Бройдена: x = {round(response['x'], 4)}, x0 = {x0}, количество итераций = {response['depth']}")
x0 = 10
response = newton_broyden_method(x0, eps, coeff)
print(f"Метод секущих: x = {round(response['x'], 4)}, x0 = {x0}, количество итераций = {response['depth']}")

Метод Ньютона: x = 2.0994, x0 = 3, количество итераций = 5
Упрощённый метод Ньютона: x = 2.0994, x0 = 10, количество итераций = 2997
Метод Ньютона-Бройдена: x = 2.0995, x0 = 3, количество итераций = 6
Метод секущих: x = 2.0994, x0 = 10, количество итераций = 12
